<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/CC_BY.png"><br />

Created by Amy Kirchhoff for [Constellate](https://constellate.orng/) under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/)<br />
For questions/comments/improvements, email amy.kirchhoff@ithaka.org.<br />

# Create a CSV of JSTOR Item UUIDs from a Dataset

(useful for feeding into Bryan Ryder's script to pull out references.)



___

## Import your dataset

We'll use the `constellate` client to automatically retrieve the dataset in the JSON file format. 

Enter a [dataset ID](https://docs.constellate.org/key-terms/#dataset-ID) in the next code cell.

If you don't have a dataset ID, you can:
* Use the sample dataset ID already in the code cell
* [Create a new dataset](https://constellate.org/builder)
* [Use a dataset ID from other pre-built sample datasets](https://constellate.org/dataset/dashboard)

In [ ]:
# Creating a variable `dataset_id` to hold our dataset ID
# The default dataset is Shakespeare Quarterly, 1950-present
dataset_id = "b7884498-03bb-277c-a031-5c4d8b014fe5"

Next, import the `constellate` client, passing the `dataset_id` as an argument using the `get_dataset` method.

In [ ]:
# Importing your dataset with a dataset ID
import constellate
# Pull in the sampled dataset (1500 documents) that matches `dataset_id`
# in the form of a gzipped JSON lines file.
# The .get_dataset() method downloads the gzipped JSONL file
# to the /data folder and returns a string for the file name and location
#dataset_file = constellate.get_dataset(dataset_id)

# To download the full dataset (up to a limit of 25,000 documents),
# request it first in the builder environment. See the Constellate Client
# documentation at: https://constellate.org/docs/constellate-client
# Then use the `constellate.download` method show below.
dataset_file = constellate.download(dataset_id, 'jsonl')

Import some of the packages we will need.

In [ ]:
import csv
import pandas as pd

Set up the dataframe

In [ ]:
df = pd.DataFrame(columns = ['id', 'doi', 'uuid' ])

Read each document using the tdm_client.dataset_reader and insert the pertinent fields into our dataframe.

In [ ]:
i=0;
for document in constellate.dataset_reader(dataset_file):
    #print(type(document))
    i=i+1;
    
    doi = document.get('doi', "")
    
    identifiers = document.get('identifier', [])
    
    #print(identifiers)
    
    local_uuid=""
    
    for id in identifiers:
        if id['name']=="local_uuid":
            local_uuid = id['value']
        if id['name']=="local_doi" and doi=="":
            doi=id['value']   
    
    df=pd.concat( [df, pd.DataFrame.from_records([{'id' : document['id'], 
                                                   'uuid': local_uuid,
                                                   'doi': doi,
                                                   'query': "select * from content_reference.grobid_references_rollup where doi='" + doi + "'"}])
                  ]
                )
    
   # print(df)
    
    if i%100 == 0:
        print("At document #: " + str(i))
        df.head()
    
        
    

In [ ]:
df.to_csv(dataset_id + "-uuids.csv", encoding='utf-8')